In [1]:
import urllib
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier


In [2]:
for x in range(2013,2021):
    f = open(f"nov{x}-{x+1}.txt", "r")
    content = f.read()
    f.close()

    content = content.replace(" ","")

    g = open(f"nov{x}-{x+1}.txt", "w")
    g.write(content)
    g.close()

In [3]:
frames = []

for x in range(2013,2021):
    frames.append(pd.read_csv(f"nov{x}-{x+1}.txt", sep = ","))

data = pd.concat(frames)

In [4]:
data.sample(10)

,Aeg,Temperatuur,Niiskus,&Otilde;hur&otilde;hk,Tuulekiirus,Tuulesuund,Sademed,UVindeks,Valgustatus,Kiirgusvoog,Radioaktiivsus
86292,2019-09-0517:40:00,22.117189,50.923297,1010.762967,4.155100,207.021909,0.0,NaN,NaN,262.906144,NaN
40837,2020-04-0203:20:00,1.373669,83.120312,997.305500,3.256271,235.877875,0.0,NaN,NaN,5.308815,NaN
30702,2020-02-2621:45:00,0.012449,98.830748,991.668100,2.894849,54.868630,0.0,NaN,NaN,3.984972,NaN
31225,2018-03-0310:00:00,-10.565714,75.778462,1011.218533,1.877822,102.224092,0.0,NaN,NaN,288.508964,NaN
53157,2014-05-1819:50:00,24.243408,47.887182,1010.138984,1.987999,108.871096,0.0,NaN,NaN,NaN,0.097779
11323,2016-12-2407:25:00,2.502665,86.564156,1011.457967,5.688861,227.215513,0.0,NaN,NaN,1.295730,NaN
36492,2016-03-2016:05:00,-2.447753,85.291820,1005.057967,2.639671,269.272118,0.0,NaN,NaN,250.174075,NaN
32365,2014-03-0709:50:00,2.590826,56.563397,1020.716526,2.178459,218.237850,0.0,NaN,NaN,NaN,0.102173
90811,2017-09-2608:25:00,10.088674,96.660981,1038.751833,3.454768,87.678772,0.0,NaN,NaN,108.507552,NaN
17638,2014-01-1506:30:00,-10.839015,64.213459,1008.324362,0.304561,96.460461,0.0,NaN,NaN,NaN,0.098899


In [5]:
data.shape

(811086, 11)

In [6]:
#f = open("kiirgusvoog_5a.csv", "r")
#content = f.read()
#f.close()

#content = content.replace(" ","")

#g = open("kiirgusvoog_5a_2.csv", "w")
#g.write(content)
#g.close()

#data = pd.read_csv("kiirgusvoog_5a_2.csv")

In [7]:
data = data.rename(columns = {'&Otilde;hur&otilde;hk':'Õhurõhk'})

In [8]:
def break_up_time(data):
    data["Aasta"]= data.apply(lambda row: row["Aeg"][0:4], axis=1)
    data["Kuu"] = data.apply(lambda row: row["Aeg"][5:7], axis=1)
    data["Päev"] = data.apply(lambda row: row["Aeg"][8:10], axis=1)
    data["Tund"] = data.apply(lambda row: row["Aeg"][10:12], axis=1)
    data["Minut"] = data.apply(lambda row: row["Aeg"][13:15], axis=1)
    data["Aeg"] = data.apply(lambda row: row["Aeg"][0:10], axis=1) + " " + data.apply(lambda row: row["Aeg"][10:18], axis=1)
    data.Aasta = data.Aasta.astype('int')
    data.Kuu = data.Kuu.astype('int')
    data.Päev = data.Päev.astype('int')
    data.Tund = data.Tund.astype('int')
    data.Minut = data.Minut.astype('int')

In [9]:
break_up_time(data)
data.head()

,Aeg,Temperatuur,Niiskus,Õhurõhk,Tuulekiirus,Tuulesuund,Sademed,UVindeks,Valgustatus,Kiirgusvoog,Radioaktiivsus,Aasta,Kuu,Päev,Tund,Minut
0,2013-11-15 00:00:00,2.253725,80.280623,1012.974940,1.396552,219.132229,0.0,NaN,NaN,NaN,0.097777,2013,11,15,0,0
1,2013-11-15 00:05:00,2.269765,80.633583,1013.042850,1.246345,220.840096,0.0,NaN,NaN,NaN,0.134524,2013,11,15,0,5
2,2013-11-15 00:10:00,2.259058,80.844891,1012.746232,1.387465,218.213758,0.0,NaN,NaN,NaN,0.091138,2013,11,15,0,10
3,2013-11-15 00:15:00,2.222720,80.999738,1012.925478,1.803502,224.199997,0.0,NaN,NaN,NaN,0.097796,2013,11,15,0,15
4,2013-11-15 00:20:00,2.135194,81.082619,1012.874290,1.640076,225.130637,0.0,NaN,NaN,NaN,0.096651,2013,11,15,0,20


In [10]:
data.columns

Index(['Aeg', 'Temperatuur', 'Niiskus', 'Õhurõhk', 'Tuulekiirus', 'Tuulesuund',
       'Sademed', 'UVindeks', 'Valgustatus', 'Kiirgusvoog', 'Radioaktiivsus',
       'Aasta', 'Kuu', 'Päev', 'Tund', 'Minut'],
      dtype='object')

In [11]:
from datetime import datetime

data["Aeg"] = data.apply(lambda row: datetime.strptime(row["Aeg"],"%Y-%m-%d %H:%M:%S"), axis=1)

In [12]:
data.dtypes

Aeg               datetime64[ns]
Temperatuur              float64
Niiskus                  float64
Õhurõhk                  float64
Tuulekiirus              float64
Tuulesuund               float64
Sademed                  float64
UVindeks                 float64
Valgustatus              float64
Kiirgusvoog              float64
Radioaktiivsus           float64
Aasta                      int64
Kuu                        int64
Päev                       int64
Tund                       int64
Minut                      int64
dtype: object

In [13]:
data = data[["Aeg","Aasta","Kuu","Päev","Tund","Minut","Temperatuur","Niiskus","Õhurõhk","Tuulekiirus","Tuulesuund","Sademed","Kiirgusvoog"]] # "Sadanudlumi"
#data['Sadanudlumi'] = data['Sadanudlumi'].fillna(0)
#data.columns = ["Aasta","Kuu","Päev","Tund","Minut","Temperatuur","Niiskus","Õhurõhk","Tuule kiirus","Tuule suund","Sademed","Kiirgusvoog"]
data['Sademed'] = data['Sademed'].fillna(0)
data['Tuulekiirus'] = data['Tuulekiirus'].fillna(method='backfill')
data['Tuulesuund'] = data['Tuulesuund'].fillna(method='backfill')
data['Temperatuur'] = data['Temperatuur'].fillna(method='backfill')
data['Niiskus'] = data['Niiskus'].fillna(method='backfill')
data['Õhurõhk'] = data['Õhurõhk'].fillna(method='backfill')
data['Kiirgusvoog'] = data['Kiirgusvoog'].fillna(method='backfill')
data["KiirgusvoogTulevikus"] = data["Kiirgusvoog"].shift(-1)
data = data.dropna()
data.isnull().sum()

Aeg                     0
Aasta                   0
Kuu                     0
Päev                    0
Tund                    0
Minut                   0
Temperatuur             0
Niiskus                 0
Õhurõhk                 0
Tuulekiirus             0
Tuulesuund              0
Sademed                 0
Kiirgusvoog             0
KiirgusvoogTulevikus    0
dtype: int64

In [14]:
data = data.rename(columns = {
    'Aeg': 'timestamp',
    'Aasta':'year',
    'Kuu':'month',
    'Päev':'day',
    'Tund':'hour',
    'Minut':'minute',
    'Temperatuur':'temperature',
    'Niiskus':'humidity',
    'Õhurõhk':'atmospheric_pressure',
    'Tuulekiirus':'wind_speed',
    'Tuulesuund':'wind_direction',
    'Sademed':'precipitation',
    #'Sadanudlumi':'snow',
    'Kiirgusvoog':'radiation_flux',
    'KiirgusvoogTulevikus':'rad_flux_infuture'    
    })
data.columns

Index(['timestamp', 'year', 'month', 'day', 'hour', 'minute', 'temperature',
       'humidity', 'atmospheric_pressure', 'wind_speed', 'wind_direction',
       'precipitation', 'radiation_flux', 'rad_flux_infuture'],
      dtype='object')

In [15]:
data[data.humidity == data.humidity.max()]

,timestamp,year,month,day,hour,minute,temperature,humidity,atmospheric_pressure,wind_speed,wind_direction,precipitation,radiation_flux,rad_flux_infuture
51611,2014-05-13 11:00:00,2014,5,13,11,0,8.624431,117.297204,990.984611,3.325451,290.185328,0.0,0.0,0.0
51611,2014-05-13 11:00:00,2014,5,13,11,0,8.624431,117.297204,990.984611,3.325451,290.185328,0.0,0.0,0.0


In [16]:
data.describe()

,year,month,day,hour,minute,temperature,humidity,atmospheric_pressure,wind_speed,wind_direction,precipitation,radiation_flux,rad_flux_infuture
count,811085.000000,811085.000000,811085.000000,811085.000000,811085.000000,811085.000000,811085.000000,811085.000000,811085.000000,811085.000000,811085.000000,811085.000000,811085.000000
mean,2017.398950,6.494621,15.751352,11.481656,27.512518,6.825025,75.458473,1010.824081,3.258511,199.267411,0.004576,89.464947,89.464947
std,2.424143,3.526065,8.820120,6.926916,17.262782,8.905742,19.869568,11.040802,1.678265,91.412932,0.050455,181.339887,181.339887
min,2013.000000,1.000000,1.000000,0.000000,0.000000,-24.307660,0.000000,963.557524,0.000000,0.000000,0.000000,-2.625525,-2.625525
25%,2016.000000,3.000000,8.000000,5.000000,14.000000,0.635374,62.394290,1003.885773,2.053779,124.554193,0.000000,0.000000,0.000000
50%,2018.000000,6.000000,16.000000,11.000000,30.000000,5.843931,81.315327,1011.116467,3.025117,224.132079,0.000000,2.639460,2.639460
75%,2019.000000,10.000000,23.000000,17.000000,45.000000,13.633509,91.499743,1018.149000,4.240506,274.416061,0.000000,74.003317,74.003317
max,2021.000000,12.000000,31.000000,23.000000,59.000000,33.553939,117.297204,1045.714767,15.497177,360.000000,9.799000,1132.409778,1132.409778


In [17]:
data.sample(50)

,timestamp,year,month,day,hour,minute,temperature,humidity,atmospheric_pressure,wind_speed,wind_direction,precipitation,radiation_flux,rad_flux_infuture
85160,2020-09-03 00:55:00,2020,9,3,0,55,14.450767,90.789945,1019.357000,4.096547,107.438462,0.000,2.314647,2.298738
104430,2021-11-04 05:45:00,2021,11,4,5,45,7.672137,92.315725,1006.785767,5.583752,238.056744,0.000,0.000000,0.000000
85140,2014-11-06 10:10:00,2014,11,6,10,10,3.665989,100.000000,993.311166,3.299492,100.081505,0.000,0.000000,0.000000
13923,2018-12-28 09:55:00,2018,12,28,9,55,0.286385,99.061362,1009.665067,1.753249,11.969612,0.000,17.526881,19.630322
103158,2016-11-07 04:25:00,2016,11,7,4,25,-5.879185,88.001696,1004.387033,5.158434,71.421159,0.000,0.642133,0.772312
15073,2017-01-06 07:55:00,2017,1,6,7,55,-16.569495,84.422659,1033.910400,0.742442,300.639450,0.000,4.130281,3.952035
77001,2020-08-05 17:00:00,2020,8,5,17,0,21.376143,47.535262,1016.893267,3.820585,273.577013,0.000,490.245492,551.224635
28134,2019-02-15 18:10:00,2019,2,15,18,10,1.934255,96.320361,1017.601900,5.921128,274.787857,0.000,2.243099,2.212167
3559,2020-11-27 08:35:00,2020,11,27,8,35,0.282641,89.676727,1007.137267,2.917003,333.421187,0.000,1.764473,2.698636
33668,2014-03-11 22:25:00,2014,3,11,22,25,1.880598,45.385813,1019.162503,4.044369,272.266887,0.000,0.000000,0.000000


In [18]:
data.head(30)

,timestamp,year,month,day,hour,minute,temperature,humidity,atmospheric_pressure,wind_speed,wind_direction,precipitation,radiation_flux,rad_flux_infuture
0,2013-11-15 00:00:00,2013,11,15,0,0,2.253725,80.280623,1012.974940,1.396552,219.132229,0.0,0.0,0.0
1,2013-11-15 00:05:00,2013,11,15,0,5,2.269765,80.633583,1013.042850,1.246345,220.840096,0.0,0.0,0.0
2,2013-11-15 00:10:00,2013,11,15,0,10,2.259058,80.844891,1012.746232,1.387465,218.213758,0.0,0.0,0.0
3,2013-11-15 00:15:00,2013,11,15,0,15,2.222720,80.999738,1012.925478,1.803502,224.199997,0.0,0.0,0.0
4,2013-11-15 00:20:00,2013,11,15,0,20,2.135194,81.082619,1012.874290,1.640076,225.130637,0.0,0.0,0.0
5,2013-11-15 00:25:00,2013,11,15,0,25,2.075500,81.316279,1012.867377,1.369254,221.732026,0.0,0.0,0.0
6,2013-11-15 00:30:00,2013,11,15,0,30,2.041885,81.697612,1012.920600,1.314422,223.314140,0.0,0.0,0.0
7,2013-11-15 00:35:00,2013,11,15,0,35,2.038886,82.163594,1013.180392,1.228393,221.865827,0.0,0.0,0.0
8,2013-11-15 00:40:00,2013,11,15,0,40,2.022422,82.593353,1013.001292,1.528448,219.857440,0.0,0.0,0.0
9,2013-11-15 00:45:00,2013,11,15,0,45,2.020750,83.062526,1012.941358,1.403501,223.718700,0.0,0.0,0.0


In [19]:
data.to_csv("data.csv",index=False)

## trying to get data directly from web page

In [18]:
#url = 'https://meteo.physic.ut.ee/et/archive.php?do=data&begin%5Byear%5D=2016&begin%5Bmon%5D=11&begin%5Bmday%5D=18&end%5Byear%5D=2021&end%5Bmon%5D=11&end%5Bmday%5D=18&9=1&12=1&10=1&15=1&16=1&14=1&snow_16=1&ok=+Esita+päring+'


def get_weather_data(start_year, start_month, start_day, end_year, end_month, end_day):
    url = f'https://meteo.physic.ut.ee/et/archive.php?do=data&begin%5Byear%5D={start_year}&begin%5Bmon%5D={start_month}&begin%5Bmday%5D={start_day}&end%5Byear%5D={end_year}&end%5Bmon%5D={end_month}&end%5Bmday%5D={end_day}&9=1&12=1&11=1&14=1&ok=+Esita+p%C3%A4ring+'
    c = pd.read_csv(url)
    #c.columns = ['Aeg', 'Temperatuur', 'Niiskus', 'Valgustatus', 'Kiirgusvoog']
    return c

In [19]:
train = get_weather_data(2011, 1, 1, 2015, 12, 12)

In [20]:
train.head()

,Aeg,Temperatuur,Niiskus,Valgustatus,Kiirgusvoog
0,2011-01-01 00:00:00,-6.936343679417071,98.6784171261325,1354.9408873569798,
1,2011-01-01 00:05:00,-6.9568330698942695,98.59701577484581,1029.54357837049,
2,2011-01-01 00:10:00,-6.97364936736915,98.42287783684701,1418.1783680055298,
3,2011-01-01 00:15:00,-6.94325190753387,98.49124327899729,1294.6821264335601,
4,2011-01-01 00:20:00,-6.86547316481287,98.64064185206159,1374.1625144454501,


In [21]:
train.isnull().sum()

Aeg             0
 Temperatuur    0
 Niiskus        0
 Valgustatus    0
 Kiirgusvoog    0
dtype: int64

# Solcast.com

In [2]:
data = pd.read_csv("Solcast_PT5M.csv")

In [3]:
data.sample(30)

,PeriodEnd,PeriodStart,Period,AirTemp,AlbedoDaily,Azimuth,CloudOpacity,DewpointTemp,Dhi,Dni,...,Ghi,GtiFixedTilt,GtiTracking,PrecipitableWater,RelativeHumidity,SnowDepth,SurfacePressure,WindDirection10m,WindSpeed10m,Zenith
537384,2012-02-09T22:10:00Z,2012-02-09T22:05:00Z,PT5M,-4.7,0.18,40,0.0,-7.2,0,0,...,0,0,0,3.9,82.2,0.5,1039.3,69,3.1,137
627587,2012-12-19T03:05:00Z,2012-12-19T03:00:00Z,PT5M,4.1,0.13,-74,0.0,3.9,0,0,...,0,0,0,10.5,98.6,0.0,1020.5,91,0.2,132
272008,2009-08-02T11:30:00Z,2009-08-02T11:25:00Z,PT5M,17.5,0.16,-172,77.7,15.5,177,0,...,177,158,177,29.8,88.5,0.0,1010.2,295,5.2,34
957487,2016-02-07T14:45:00Z,2016-02-07T14:40:00Z,PT5M,8.9,0.13,138,0.0,2.7,57,588,...,190,490,159,9.9,64.9,0.0,998.7,220,9.3,77
221637,2009-02-08T13:55:00Z,2009-02-08T13:50:00Z,PT5M,3.0,0.15,150,62.3,0.4,106,0,...,106,94,106,7.5,83.0,0.0,1003.2,261,5.0,72
265458,2009-07-10T17:40:00Z,2009-07-10T17:35:00Z,PT5M,16.1,0.15,77,56.3,9.9,108,0,...,108,97,108,19.8,67.0,0.0,1014.5,294,6.3,71
665749,2013-04-30T15:15:00Z,2013-04-30T15:10:00Z,PT5M,13.1,0.15,111,0.0,3.1,123,730,...,522,578,869,10.3,50.7,0.0,1023.8,21,5.4,57
365464,2010-06-22T23:30:00Z,2010-06-22T23:25:00Z,PT5M,10.9,0.16,3,0.0,9.0,0,0,...,0,0,0,19.8,88.6,0.0,1023.8,64,2.2,104
2519,2007-01-09T18:05:00Z,2007-01-09T18:00:00Z,PT5M,12.5,0.16,101,30.9,7.9,0,0,...,0,0,0,17.6,73.5,0.0,1009.6,227,10.8,110
211676,2009-01-04T23:50:00Z,2009-01-04T23:45:00Z,PT5M,1.2,0.12,-2,97.0,0.9,0,0,...,0,0,0,11.3,98.1,1.3,1014.7,274,3.1,151
